![OpenSARlab notebook banner](NotebookAddons/blackboard-banner.png)

# Exercise8A-InSARTimeSeriesGIAnTPreparation

<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" />

### Franz J Meyer & Joshua J C Knicely; University of Alaska Fairbanks

This notebook accompanies Exercise8B-InSARTimeSeriesGIANTProcessing.ipynb

The aim is to provide you background on how you can prepare interferograms so GIAnT can process them. 

This notebook uses a dataset that was prepared in advance. **You do not need to follow or execute the code contained in this notebook.** It is designed to provide the background on how to prepare interferograms for processing in GIAnT so that you can generate an InSAR time series using your own dataset if desired.

<br>
<img style="padding:7px;" src="NotebookAddons/OpenSARlab_logo.svg" width="170" align="right" />

The code blocks show what was used to generate the files prepared for the InSARTimeSeriesGIANTProcessing notebook, and they would need to be amended as appropriate to process a different InSAR stack. The code is provided for your reference, but cannot be used directly as written to prepare a different dataset.

---
**Important Note about JupyterHub**

Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/insar_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "insar_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "insar_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "insar_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

## 1. Import Python Libraries:

**Import the Python libraries and modules we will need to run this lab:**

In [ ]:
from datetime import date
from pathlib import Path
import zipfile
import h5py # for is_hdf5
import shutil

from osgeo import osr
from osgeo import gdal
import pandas as pd 
import numpy as np

from IPython.display import HTML

from opensarlab_lib import asf_unzip

**Download GIAnT from the asf-jupyter-data S3 bucket**

GIAnT is no longer supported (Python 2). This unofficial version of GIAnT has been partially ported to Python 3 to run this notebook. Only the portions of GIAnT used in this notebook have been tested.

In [ ]:
giant_path = Path("/home/jovyan/.local/GIAnT/SCR")

if not giant_path.parent.exists():
    download_path = 's3://asf-jupyter-data-west/GIAnT_5_21.zip'
    output_path = f"/home/jovyan/.local/{Path(download_path).name}"
    !aws --region=us-west-2 --no-sign-request s3 cp $download_path $output_path
    if Path(output_path).is_file():
        !unzip $output_path -d /home/jovyan/.local/
        Path(output_path).unlink()

## 2. Create Input Files And Code for GIAnT

The code below shows how to create the input files and specialty code that GIAnT requires. For this lab, 'ifg.list' is not needed, 'date.mli.par' has already been provided, 'prepxml_SBAS.py' is not needed as the 'sbas.xml' and 'data.xml' files it would create have already been provided, and 'userfn.py' is not needed as we are skipping the step in which it would be used.

The files that would be created are listed below:
        
- ifg.list
    - List of the interferogram properties including master and slave date, perpendicular baseline, and sensor. 
- date.mli.par
    - File from which GIAnT pulls requisite information about the sensor. 
    - This is specifically for GAMMA files. When using other interferogram processing techniques, an alternate file is required. 
- prepxml_SBAS.py
    - Python function to create an xml file that specifies the processing options to GIAnT. 
    - This must be modified by the user for their particular application. 
- userfn.py
    - Python function to map the interferogram dates to a phyiscal file on disk. 
    - This must be modified by the user for their particular application. 
    
---
    
### 2.1 Create 'ifg.list' File

This will create simple 4 column text file will communicate network information to GIAnT. It will be created within the `GIAnT` directory.

**This step has already been done, so we will not actually create the 'ifg.list' file. This code is displayed for your potential future use.**

In [ ]:
"""
# Get one of each file name. This assumes the unwrapped phase geotiff has been converted to a '.flt' file
files = [f for f in os.listdir(datadirectory) if f.endswith('_unw_phase.flt')] 

# Get all of the master and slave dates. 
masterDates,slaveDates = [],[]
for file in files:
    masterDates.append(file[0:8])
    slaveDates.append(file[9:17])
# Sort the dates according to the master dates. 
master_dates,sDates = (list(t) for t in zip(*sorted(zip(masterDates,slaveDates))))

with open( os.path.join('GIAnT', 'ifg.list'), 'w') as fid:
    for i in range(len(master_dates)):
        masterDate = master_dates[i] # pull out master Date (first set of numbers)
        slaveDate = sDates[i] # pull out slave Date (second set of numbers)
        bperp = '0.0' # according to JPL notebooks
        sensor = 'S1' # according to JPL notebooks
        fid.write(f'{masterDate}  {slaveDate}  {bperp}  {sensor}\n') # write values to the 'ifg.list' file. 
"""

*You may notice that the code above sets the perpendicular baseline to a value of 0.0 m. This is not the true perpendicular baseline. That value can be found in metadata file (titled '$<$master timestamp$>$_$<$slave timestamp$>$.txt') that comes with the original interferogram. Generally, we would want the true baseline for each interferogram. However, since Sentinel-1 has such a short baseline, a value of 0.0 m is sufficient for our purposes.*

### 2.2 Create 'date.mli.par' File

As we are using GAMMA products, we must create a 'date.mli.par' file from which GIAnT will pull necessary information. If another processing technique is used to create the interferograms, an alternate file name and file inputs are required.

**Again, this step has already been completed and the code is only displayed for your potential future use.**

In [ ]:
"""
# Create file 'date.mli.par'

# Get file names
files = [f for f in os.listdir(datadirectory) if f.endswith('_unw_phase.flt')]

# Get WIDTH (xsize) and FILE_LENGTH (ysize) information
ds = gdal.Open(datadirectory+files[0], gdal.GA_ReadOnly)
type(ds)

nLines = ds.RasterYSize
nPixels = ds.RasterXSize

trans = ds.GetGeoTransform()
ds = None

# Get the center line UTC time stamp; can also be found inside <date>_<date>.txt file and hard coded
dirName = os.listdir('ingrams')[0] # get original file name (any file can be used; the timestamps are different by a few seconds)
vals = dirName.split('-') # break file name into parts using the separator '-'
tstamp = vals[2][9:16] # extract the time stamp from the 2nd datetime (could be the first)
c_l_utc = int(tstamp[0:2])*3600 + int(tstamp[2:4])*60 + int(tstamp[4:6])

rfreq = 299792548.0 / 0.055465763 # radar frequency; speed of light divided by radar wavelength of Sentinel1 in meters

# write the 'date.mli.par' file
with open(os.path.join(path, 'date.mli.par'), 'w') as fid:
    # Method 1
    fid.write(f'radar_frequency: {rfreq} \n') # when using GAMMA products, GIAnT requires the radar frequency. Everything else is in wavelength (m) 
    fid.write(f'center_time: {c_l_utc} \n') # Method from Tom Logan's prepGIAnT code; can also be found inside <date>_<date>.txt file and hard coded
    fid.write( 'heading: -11.9617913 \n') # inside <date>_<date>.txt file; can be hardcoded or set up so code finds it. 
    fid.write(f'azimuth_lines: {nLines} \n') # number of lines in direction of the satellite's flight path
    fid.write(f'range_samples: {nPixels} \n') # number of pixels in direction perpendicular to satellite's flight path
    fid.close() # close the file
"""

### 2.3 Make prepxml_SBAS.py

We will create a prepxml_SBAS.py function and put it into our GIAnT working directory. Again, this is shown for anyone that may want to use GIAnT on their own.

If we do wish to change 'sbas.xml' or 'data.xml', this can be done by creating and running a new 'prepxml_SBAS.py'.

### 2.3.1 Necessary prepxml_SBAS.py edits

GIAnT comes with an example prepxml_SBAS.py, but requries significant edits for our purposes. These alterations have already been made, so we don't have to do anything now, but it is good to know the kinds of things that have to be altered. The details of some of these options can be found in the GIAnT documentation. The rest must be found in the GIAnT processing files themselves, most notably the tsxml.py and tsio.py functions.

The following alterations were made:

- Changed 'example' &#9658; 'date.mli.par'
- Removed 'xlim', 'ylim', 'ref_x_lim', and 'ref_y_lim'
    - These are used for clipping the files in GIAnT. As we have already done this, it is not necessary. 
- Removed latfile='lat.map' and lonfile='lon.map'
    - These are optional inputs for the latitude and longitude maps. 
- Removed hgtfile='hgt.map'
    - This is an optional altitude file for the sensor. 
- Removed inc=21.
    - This is the optional incidence angle information. 
    - It can be a constant float value or incidence angle file. 
    - For Sentinel1, it varies from 29.1-46.0&deg;.
- Removed masktype='f4'
    - This is the mask designation. 
    - We are not using any masks for this. 
- Changed unwfmt='RMG' &#9658; unwfmt='GRD'
    - Read data using GDAL. 
- Removed demfmt='RMG'
- Changed corfmt='RMG' &#9658; corfmt='GRD'
    - Read data using GDAL. 
- Changed nvalid=30 -> nvalid=1
    - This is the minimum number of interferograms in which a pixel must be coherent. A particular pixel will be included only if its coherence is above the coherence threshold, cohth, in more than nvalid number of interferograms. 
- Removed atmos='ECMWF'
    - This is an amtospheric correction command. It depends on a library called 'pyaps' developed for GIAnT. This library has not been installed yet. 
- Changed masterdate='19920604' &#9658; masterdate='20161119'
    - Use our actual masterdate. 
    - I simply selected the earliest date as the masterdate. 

Defining a reference region is a potentially important step. This is a region at which there should be no deformation. For a volcano, this should be some significant distance away from the volcano. GIAnT has the ability to automatically select a reference region which we will use for this exercise.

Below is an example of how the reference region would be defined when creating the XML file with the processing parameters. If we look at the prepxml_SBAS.py code below, ref_x_lim and ref_y_lim, the pixel based location of the reference region, is within the code, but has been commented out. 

**Define reference region:**

In [ ]:
ref_x_lim, ref_y_lim = [0, 10], [95, 105]

Below is an example of how the reference region would be defined. Look at the prepxml_SBAS.py code below. Note that ref_x_lim and ref_y_lim (the pixel based location of the reference region) are within the code.

**This has already been completed but the code is here as an example script for creating XML files for use with the SBAS processing chain.**

In [ ]:
'''
#!/usr/bin/env python

import tsinsar as ts
import argparse
import numpy as np

def parse():
    parser= argparse.ArgumentParser(description='Preparation of XML files for setting up the processing chain. Check tsinsar/tsxml.py for details on the parameters.')
    parser.parse_args()

parse()
g = ts.TSXML('data')
g.prepare_data_xml(
    'date.mli.par', proc='GAMMA', 
    #ref_x_lim = [{1},{2}], ref_y_lim=[{3},{4}],
    inc = 21., cohth=0.10, 
    unwfmt='GRD', corfmt='GRD', chgendian='True', endianlist=['UNW','COR'])
g.writexml('data.xml')


g = ts.TSXML('params')
g.prepare_sbas_xml(nvalid=1, netramp=True, demerr=False, uwcheck=False, regu=True, masterdate='{5}', filt=1.0)
g.writexml('sbas.xml')


############################################################
# Program is part of GIAnT v1.0                            #
# Copyright 2012, by the California Institute of Technology#
# Contact: earthdef@gps.caltech.edu                        #
############################################################

'''

**Set the master date and create a script for creating XML files for use with the SBAS processing chain:**

In [ ]:
#files = [f for f in os.listdir(datadirectory) if f.endswith('_unw_phase.flt')]
#master_date = min([files[i][0:8] for i in range(len(files))], key=int)

# master_date = '20161119'

# prepxml_SBAS_Template = '''
#!/usr/bin/env python

'''Example script for creating XML files for use with the SBAS processing chain. This script is supposed to be copied to the working directory and modified as needed."""


import tsinsar as ts
import argparse
import numpy as np

def parse():
    parser= argparse.ArgumentParser(description='Preparation of XML files for setting up the processing chain. Check tsinsar/tsxml.py for details on the parameters.')
    parser.parse_args()

parse()
g = ts.TSXML('data')
g.prepare_data_xml(
    'date.mli.par', proc='GAMMA', 
    #ref_x_lim = [{1},{2}], ref_y_lim=[{3},{4}],
    inc = 21., cohth=0.10, 
    unwfmt='GRD', corfmt='GRD', chgendian='True', endianlist=['UNW','COR'])
g.writexml('data.xml')


g = ts.TSXML('params')
g.prepare_sbas_xml(nvalid=1, netramp=True, demerr=False, uwcheck=False, regu=True, masterdate='{5}', filt=1.0)
g.writexml('sbas.xml')


############################################################
# Program is part of GIAnT v1.0                            #
# Copyright 2012, by the California Institute of Technology#
# Contact: earthdef@gps.caltech.edu                        #
############################################################

'''
#with open(os.path.join(path,'prepxml_SBAS.py'), 'w') as fid:
#    fid.write(prepxml_SBAS_Template.format(path,ref_x_lim[0],ref_x_lim[1],ref_y_lim[0],ref_y_lim[1],master_date))

To create a new 'sbas.xml' and 'data.xml' file, we would modify the above code to give new parameters and to write to the appropriate folder (e.g., to change the time filter from 1 year to none and to write to the directory in which we are working; 'filt=1.0' -> 'filt=0.0'; and 'os.path.join(path,'prepxml_SBAS.py') -> 'prepxml_SBAS.py' OR '%cd ~' into your home directory). Then we would run it below.

### 2.4 Run prepxml_SBAS.py

Here we run **prepxml_SBAS.py** to create the 2 needed files

- data.xml 
- sbas.xml

To use MinTS, we would run **prepxml_MinTS.py** to create

- data.xml
- mints.xml
        
These files are needed by **PrepIgramStack.py**.

**Run prepxml_SBAS.py in python 2.7 and check the output to confirm that your input values are correct:**

In [ ]:
# !python $giant_path/prepxml_SBAS.py # this has already been done. data.xml and sbas.xml already exist

**Make sure the two requisite xml files (data.xml and sbas.xml) were produced after running prepxml_SBAS.py.**

**Display the contents of data.xml:**

In [ ]:
data_x_path = next(giant_path.parent.rglob('data.xml'))

if data_x_path.exists():
    !cat {data_x_path}

**Display the contents of sbas.xml:**

In [ ]:
sbas_x_path = next(giant_path.parent.rglob('sbas.xml'))

if sbas_x_path.exists():
    !cat {sbas_x_path}

### 2.5 Create userfn.py

Before running the next piece of code, `PrepIgramStack.py`, we must create a python file called `userfn.py`. This file maps the interferogram dates to a physical file on disk. This python file must be in our working directory, `/GIAnT`. We can create this file from within the notebook using python. 

**Again, this step has already been preformed and is unnecessary, but the code is provided as an example.**

In [ ]:
'''
userfnTemplate = """
#!/usr/bin/env python
import os 

def makefnames(dates1, dates2, sensor):
    dirname = '{0}'
    root = os.path.join(dirname, dates1+'-'+dates2)
    #unwname = root+'_unw_phase.flt' # for potentially disruptive default values kept. 
    unwname = root+'_unw_phase_no_default.flt' # for potentially disruptive default values removed. 
    corname = root+'_corr.flt'
    return unwname, corname
"""

with open('userfn.py', 'w') as fid:
    fid.write(userfnTemplate.format(path))
'''

*Exercise8A-InSARTimeSeriesGIANTPreparation.ipynb - Version 1.3 - November 2021*

*Version Changes:*

- *asf_notebook -> opensarlab_lib*
- *url_widget*
- *html -> markdown*